# JDBC Read & Write

#### 이번 Lab에서는 Glue Job을 이용해 Database에 접근하고 Read, Write하는 예제를 살펴봅니다.

#### ACCOUNT_ID와 RDS의 HOST, USER, PASSWD 정보를 입력합니다.

In [ ]:
ACCOUNT_ID = ''
HOST = ''
DATABASE = 'analytics_hol'
TABLE = 'titanic_train'
JDBC_URL = 'jdbc:mysql://{HOST}:3306/{DATABASE}'.format(HOST=HOST, DATABASE=DATABASE)
USER = 'admin'
PASSWD = ''

#### Lab을 진행하기 앞서 필요한 Database와 Table을 생성합니다.

In [ ]:
import pymysql

conn = pymysql.connect(host=HOST, user=USER, passwd=PASSWD, connect_timeout=5)   

try:
    with conn.cursor() as cursor:
        # drop db
        query = 'DROP DATABASE IF EXISTS {DATABASE}'.format(DATABASE=DATABASE)
        cursor.execute(query)
        
        # create db
        query = 'CREATE DATABASE IF NOT EXISTS {DATABASE}'.format(DATABASE=DATABASE)
        cursor.execute(query)
        
        # create table
        query = '''
CREATE TABLE IF NOT EXISTS {DATABASE}.{TABLE} (
  `passengerid` int(11) NOT NULL,
  `survived` tinyint(1) DEFAULT NULL,
  `pclass` tinyint(4) DEFAULT NULL,
  `name` varchar(128) COLLATE utf8_bin DEFAULT NULL,
  `sex` char(8) DEFAULT NULL,
  `age` tinyint(4) DEFAULT NULL,
  `sibsp` tinyint(4) DEFAULT NULL,
  `parch` tinyint(4) DEFAULT NULL,
  `ticket` varchar(32) COLLATE utf8_bin DEFAULT NULL,
  `fare` DECIMAL(10, 6) DEFAULT NULL,
  `cabin` varchar(32) COLLATE utf8_bin DEFAULT NULL,
  `embarked` char(1) DEFAULT NULL,
  `created_time` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
  `updated_time` timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
  PRIMARY KEY (`passengerid`),
  KEY `survived` (`survived`),
  KEY `pclass` (`pclass`),
  KEY `sex` (`sex`),
  KEY `embarked` (`embarked`),
  KEY `created_time` (`created_time`),
  KEY `updated_time` (`updated_time`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
        '''.format(DATABASE=DATABASE, TABLE=TABLE)
        cursor.execute(query)

        conn.commit()
except Exception as e:
    print('[ERROR]: {}'.format(e))
    raise
finally:
    conn.close()

#### Lab Guide로 돌아가서 Glue Connection을 생성합니다.

#### JDBC Write with Glue & Spark API

In [ ]:
from awsglue.context import GlueContext

# GlueContext 생성
glueContext = GlueContext(sc)

s3_bucket = 's3://aws-glue-hol-' + ACCOUNT_ID

# Read Data from Glue Catalog(S3)
titanic_dyf = glueContext.create_dynamic_frame.from_catalog(database=DATABASE,
                                                           table_name=TABLE,                           
                                                           transformation_ctx='titanic_dyf')
# Write Data into Glue Catalog(JDBC)
datasink = glueContext.write_dynamic_frame.from_catalog(
    frame = titanic_dyf, 
    database = DATABASE, 
    table_name = '_'.join([DATABASE, TABLE]))

# Spark API
# JDBC Connection 정보
#connectionProperties = {    
#    "user" : USER,
#    "password" : PASSWD,
#    "driver" : "com.mysql.jdbc.Driver"
#}

#titanic_dyf.toDF().write.jdbc(
#    url=JDBC_URL, 
#    table=TABLE, 
#    mode="overwrite", 
#    properties=connectionProperties
#)

#### JDBC Read with Glue API

In [ ]:
from awsglue.context import GlueContext

# GlueContext 생성
glueContext = GlueContext(sc)

# Read Data from Glue Catalog(JDBC)
titanic_dyf = glueContext.create_dynamic_frame.from_catalog(database=DATABASE,
                                                           table_name='_'.join([DATABASE, TABLE]),                           
                                                           transformation_ctx='titanic_dyf',
                                                           additional_options={'hashexpression': 'passengerid', 
                                                                               'hashpartitions': 5})

titanic_dyf.toDF().show()

#### JDBC Read with Spark API

In [ ]:
from awsglue.context import GlueContext

# JDBC 접속 정보
connectionProperties = {
    "user" : USER,
    "password" : PASSWD,
    "driver" : "com.mysql.jdbc.Driver",
    "fetchsize" : "1000"
}

# pushdown_query를 이용해 부분 데이터만 읽어올 수 있다.
pushdown_query = "(select * from {TABLE} where passengerid < 100) {TABLE}_alias".format(TABLE=TABLE)

# lowerBound, upperBound, numPartitions은 Partition을 나누는 기준을 정할 때 사용되며 데이터를 읽어오는 범위와는 관계없음
titanic_df = spark.read.jdbc(
        url=JDBC_URL,
        table=pushdown_query,
        column="passengerid",
        lowerBound=1,
        upperBound=100,
        numPartitions=5,
        properties=connectionProperties)

titanic_df.show()